In [1]:
import pandas

In [2]:
a = pandas.read_csv("6M Partition/1940.csv")
a.tail()

,textID,# words,genre,year,title,author,Publication information,Library of Congress classification (NF),FIXED
2697,773626,"22,744",NF,1949,Handicrafts of New England,Allen H. Eaton,"Harper and Brothers Publishers, New York",N,n
2698,774245,"40,990",NF,1949,A Short History of Science and Scientific Thought,F. Sherwood Taylor,New York: W. W. Norton & Company,Q,n
2699,774247,"42,072",NF,1949,The Origins of Modern Science,H. Butterfield,New York: The Macmillan Company,Q,n
2700,774311,"39,501",NF,1949,Clothing for Moderns,Mabel D. Erwin,New York: The MacMillan Company,T,n
2701,774345,"43,624",NF,1949,Wells Fargo: Advancing the American Frontier,Edward Hungerford,"Random House, New York",H,n


In [3]:
genre = ["FIC", "NF", "MAG"]
years = [1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000]
verb = ['make', 'take', 'give', 'bring', 'bear', 'lose', 'grant', 'create', 'hold', 'keep']

In [4]:
for i in genre:
    print(i)

FIC
NF
MAG


In [5]:
f=open("HandpickedSelectionByGenre.sql", "a+")
for v in verb:
    for i in years:
        a = pandas.read_csv("6M Partition/" + str(i) + ".csv")
        for j in genre:
            sqlQuery = ""
            sqlSource = ""
            sqlQuery = "\n" + "--" + str(i) + j + v + "\n"
            b = a.loc[a['genre'] == j]
            for row in b.iterrows():
                sqlSource = sqlSource + "s.textid = " + str(row[1][0]) + " or "
            sqlSource = sqlSource[:-3]
            sqlQuery = sqlQuery + """
COPY (
SELECT count(*), lemma from (
SELECT c.w2 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and (
""" + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w3 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w4 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w5 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w6 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """ ) a, lexicon b where 
(
b.pos like '%nn@%' or b.pos like '%nn\%%' or b.pos like 'nn'
or b.pos like '%nn1%' or b.pos like '%nn2%'
) 
and a.w2 = b.wid group by b.lemma order by count(*) desc 
) TO """ + "\'/var/lib/postgresql/HandpickedFullSelectByGenre/" + v + str(i) + j + ".csv\' CSV HEADER;"
            
            f.write(sqlQuery)
f.close()